In [1]:
### forest-disturbance-stack-v3

### This script uses several functions to create custom annual disturbance stacks. First,
### a template raster is created using a reference raster. Next, a forest mask is resampled and projected 
### to match template. Next, input disturbance rasters are subsetted to years of interest
### and resampled to match template. Finally, each input disturbance raster is stacked together,
### annually, and forest mask is applied to each stack/band.

### Returns and saves final GeoTIFF (.tif) for each year.


# 0. Setup
# 1. Template raster setup
# 2. Input rasters and metadata
# 3. Resample & Reproject Forest Mask to Template
# 4. Subset & resample input rasters to template
# 5. Create annual stacks



# ============================================================
# 0. Setup
# ============================================================

# Working directory
setwd("/home/jovyan/data-store/forest-disturbance-stack-v3")
here::i_am("README.md")

# Packages
packages <- c("here", "terra", "fs")
installed <- packages %in% installed.packages()[, "Package"]
if (any(!installed)) install.packages(packages[!installed])

library(here)
library(terra)
library(fs)

# Create new data directories
dir.create(here("data", "derived", "not_resampled"), showWarnings = FALSE, recursive = TRUE)
dir.create(here("data", "derived", "resampled"), showWarnings = FALSE, recursive = TRUE)

# Make sure input rasters and forest mask are placed in data/derived/not_resampled

# Dedicated temp directory for CyVerse (no local SSD)
cyverse_tmp <- here("tmp_terra")
dir.create(cyverse_tmp, showWarnings = FALSE, recursive = TRUE)

terraOptions(
  tempdir = cyverse_tmp,
  memfrac = 0.4,    # use 40% of RAM
  memmax = 180000,   # cap memory at 64 GB
  progress = 1,
  threads = 1       # avoid I/O contention
)
Sys.setenv(TMPDIR = cyverse_tmp)

here() starts at /home/jovyan/data-store/forest-disturbance-stack-v3

terra 1.7.55



In [2]:
# ============================================================
# 1. Template raster setup
# ============================================================

# Template path
template_path <- here("data", "derived", "resampled", "template_30m_singleband.tif")

if (!file.exists(template_path)) {
  
  message("Creating template raster ...")
  
  # Reference raster (any input raster is fine)
  ref_raster_path <- here("data", "derived", "not_resampled", "wildfire_id.tif")
  ref <- rast(ref_raster_path)
  
  # Build a single-band template for alignment
  single_template <- rast(
    ext = ext(ref),
    res = res(ref),
    crs = crs(ref),
    nlyr = 1
  )
  values(single_template) <- NA_real_
  
  # Ensure directory exists
  dir.create(dirname(template_path), showWarnings = FALSE, recursive = TRUE)
  
  # Save template
  writeRaster(
    single_template,
    template_path,
    datatype = "FLT4S",
    overwrite = TRUE,
    gdal = c("COMPRESS=LZW", "TILED=YES", "BIGTIFF=YES")
  )
  
  message("✅ Template created at: ", template_path)
  
} else {
  message("✅ Template already exists at: ", template_path, " – skipping creation")
}

✅ Template already exists at: /home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/template_30m_singleband.tif – skipping creation



In [3]:
# ============================================================
# 2. Input rasters and metadata
# ============================================================

raster_paths <- c(
  wildfire = here("data", "derived", "not_resampled", "wildfire_id.tif"),
  biotic   = here("data", "derived", "not_resampled", "biotic_gridded_1km_all_years_severity.tif"),
  pdsi     = here("data", "derived", "not_resampled", "pdsi_annual.tif"),
  hd       = here("data", "derived", "not_resampled", "hd_fingerprint.tif")
)

# Starting years of each raster (first band)
start_years <- c(wildfire = 1984, biotic = 1997, pdsi = 2000, hd = 1958)

# Forest mask
forest_mask_path <- here("data", "derived", "not_resampled", "relaxed_forest_mask_2000_2020.tif")
forest_mask_resampled <- here("data", "derived", "resampled", "forest_mask_30m_resampled.tif")

# Years of interest
years_of_interest <- 2000:2020

In [4]:
# ============================================================
# 3. Resample & Reproject Forest Mask to Template
# ============================================================

forest_mask_resampled <- here("data", "derived", "resampled", "forest_mask_30m_resampled.tif")
template_path <- here("data", "derived", "resampled", "template_30m_singleband.tif")

if (!file.exists(forest_mask_resampled)) {
  
  message("Resampling and reprojecting forest mask ...")
  
  # Load original forest mask
  mask_r <- rast(forest_mask_path)
  if (nlyr(mask_r) > 1) mask_r <- mask_r[[1]]  # single band
  
  # Load template as SpatRaster
  template_r <- rast(template_path)
  
  # Reproject + resample to match template CRS, extent, resolution
  mask_aligned <- project(mask_r, template_r, method = "near")  # nearest neighbor for categorical
  
  # Binarize: 1 = forest, 0 = non-forest
  mask_aligned <- mask_aligned > 0
  
  # Write raster safely
  writeRaster(
    mask_aligned,
    forest_mask_resampled,
    overwrite = TRUE,
    datatype = "INT1U",
    gdal = c("COMPRESS=LZW", "BIGTIFF=YES", "TILED=YES")
  )
  
  # Quick check
  mask_check <- rast(forest_mask_resampled)
  message("✅ Forest mask ready: ", forest_mask_resampled)
  message("   CRS: ", crs(mask_check))
  message("   Extent: ", ext(mask_check))
  message("   Resolution: ", res(mask_check))
  
} else {
  message("✅ Forest mask already exists at: ", forest_mask_resampled, " – skipping resampling")
}

✅ Forest mask already exists at: /home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/forest_mask_30m_resampled.tif – skipping resampling



In [5]:
# ============================================================
# 4. Subset & resample input rasters to template
# ============================================================

# Compute which bands overlap the years of interest
compute_band_indices <- function(start_year, n_bands, years_of_interest) {
  all_years <- seq(from = start_year, length.out = n_bands)
  which(all_years %in% years_of_interest)
}

# Decide resampling method based on raster type
default_resample_method <- function(file) {
  if (grepl("wildfire|hd", file, ignore.case = TRUE)) return("near")   # categorical
  return("bilinear")  # biotic, pdsi, and other continuous rasters
}

subset_then_resample <- function(template_path,
                                 raster_paths,
                                 start_years,
                                 years_of_interest,
                                 tmp_dir = "tmp_terra",
                                 n_threads = 8,
                                 overwrite = FALSE)
{
  if (!dir.exists(tmp_dir)) dir.create(tmp_dir, recursive = TRUE)
  
  # Load template raster info
  tmpl <- rast(template_path)
  ex <- ext(tmpl)
  res_x <- res(tmpl)[1]
  res_y <- res(tmpl)[2]
  
  resampled_outs <- character(length(raster_paths))
  names(resampled_outs) <- basename(raster_paths)
  
  skipped_subset <- character()
  skipped_resample <- character()
  
  for (i in seq_along(raster_paths)) {
    
    in_file <- raster_paths[[i]]
    start_year <- start_years[[i]]
    
    # Load raster to determine number of bands
    src <- rast(in_file)
    n_bands <- nlyr(src)
    rm(src); gc()
    
    # Compute band indices overlapping years of interest
    band_idx <- compute_band_indices(start_year, n_bands, years_of_interest)
    if (length(band_idx) == 0) {
      message("⚠️ No overlapping years in ", basename(in_file))
      next
    }
    
    # Define file paths for subset and resampled outputs
    base <- tools::file_path_sans_ext(basename(in_file))
    subset_file <- file.path(tmp_dir,
                             paste0(base, "_",
                                    min(years_of_interest), "_",
                                    max(years_of_interest),
                                    "_subset.tif"))
    
    resampled_file <- file.path(tmp_dir,
                                paste0(base, "_30m_resampled.tif"))
    
    # -------------------------------------------------------
    # 1. SUBSET (same for categorical and continuous)
    # -------------------------------------------------------
    if (!file.exists(subset_file) || overwrite) {
      b_args <- paste(sprintf("-b %d", band_idx), collapse = " ")
      cmd_tr <- sprintf(
        "gdal_translate %s %s %s",
        b_args, shQuote(in_file), shQuote(subset_file)
      )
      message("🪶 Subsetting bands for ", basename(in_file))
      if (system(cmd_tr) != 0) stop("gdal_translate failed for ", in_file)
    } else {
      message("✅ Subset already exists for ", basename(in_file))
      skipped_subset <- c(skipped_subset, basename(in_file))
    }
    
    # -------------------------------------------------------
    # 2. RESAMPLE
    # -------------------------------------------------------
    if (!file.exists(resampled_file) || overwrite) {
      
      # Determine method and raster type
      method <- default_resample_method(in_file)
      is_categorical <- method == "near"
      raster_type_msg <- ifelse(is_categorical, "categorical", "continuous")
      
      cmd_warp <- sprintf(
        paste(
          "gdalwarp -tr %s %s -r %s",
          "-te %s %s %s %s",
          "-t_srs EPSG:5070",
          "-multi -wo NUM_THREADS=%d",
          "-co COMPRESS=DEFLATE -co BIGTIFF=YES",
          "%s %s"
        ),
        res_x, res_y, method,
        ex[1], ex[3], ex[2], ex[4],
        n_threads,
        shQuote(subset_file), shQuote(resampled_file)
      )
      
      message("🌀 Resampling ", raster_type_msg, " ", basename(in_file))
      
      if (system(cmd_warp) != 0) stop("gdalwarp failed for ", subset_file)
      
    } else {
      message("✅ Resampling skipped for ", basename(in_file))
      skipped_resample <- c(skipped_resample, basename(in_file))
    }
    
    # Save normalized path
    resampled_outs[i] <- normalizePath(resampled_file)
  }
  
  # Summary messages
  if (length(skipped_subset) > 0)
    message("📄 Subset skipped for: ", paste(skipped_subset, collapse = ", "))
  if (length(skipped_resample) > 0)
    message("📄 Resampling skipped for: ", paste(skipped_resample, collapse = ", "))
  
  message("🎉 All resampled files are ready in: ", normalizePath(tmp_dir))
  return(resampled_outs)
}


# Run resampling
resampled_files <- subset_then_resample(
  template_path = template_path,
  raster_paths = raster_paths,
  start_years = start_years,
  years_of_interest = years_of_interest,
  overwrite = FALSE
)


✅ Subset already exists for wildfire_id.tif

✅ Resampling skipped for wildfire_id.tif

✅ Subset already exists for biotic_gridded_1km_all_years_severity.tif

✅ Resampling skipped for biotic_gridded_1km_all_years_severity.tif

✅ Subset already exists for pdsi_annual.tif

✅ Resampling skipped for pdsi_annual.tif

✅ Subset already exists for hd_fingerprint.tif

✅ Resampling skipped for hd_fingerprint.tif

📄 Subset skipped for: wildfire_id.tif, biotic_gridded_1km_all_years_severity.tif, pdsi_annual.tif, hd_fingerprint.tif

📄 Resampling skipped for: wildfire_id.tif, biotic_gridded_1km_all_years_severity.tif, pdsi_annual.tif, hd_fingerprint.tif

🎉 All resampled files are ready in: /home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra



In [6]:
# ============================================================
# 5. Move temp files to /resampled folder for GDAL command line next step
# ============================================================

# Source and destination folders
src_dir <- here("tmp_terra")
dst_dir <- here("data", "derived", "resampled")

# Make sure destination exists
dir_create(dst_dir)

# List all GeoTIFFs (.tif) in temp_terra
tifs <- dir_ls(src_dir, glob = "*.tif")

# Preview before moving (safety check)
print(tifs)

# Move files (use file_move to rename in place; or file_copy if you want to duplicate)
file_copy(tifs, dst_dir, overwrite = TRUE)

# Verify
dir_ls(dst_dir, glob = "*.tif")


/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/biotic_gridded_1km_all_years_severity_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/biotic_gridded_1km_all_years_severity_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/hd_fingerprint_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/hd_fingerprint_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/pdsi_annual_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/pdsi_annual_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/wildfire_id_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/tmp_terra/wildfire_id_30m_resampled.tif


/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/biotic_gridded_1km_all_years_severity_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/biotic_gridded_1km_all_years_severity_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/forest_mask_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/hd_fingerprint_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/hd_fingerprint_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/pdsi_annual_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/pdsi_annual_30m_resampled.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/template_30m_singleband.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/wildfire_id_2000_2020_subset.tif
/home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/resampled/wildfire_id_30m_resampled.tif

In [16]:
# ============================================================
# 6. Align mask to wildfire raster
# ============================================================

# Load the original mask
forest <- rast("data/derived/resampled/forest_mask_30m_resampled.tif")

# List of rasters you want to align the mask to
rasters_to_align <- c(
  "data/derived/resampled/wildfire_id_30m_resampled.tif",
  "data/derived/resampled/biotic_gridded_1km_all_years_severity_30m_resampled.tif",
  "data/derived/resampled/hd_fingerprint_30m_resampled.tif",
  "data/derived/resampled/pdsi_annual_30m_resampled.tif"
)

for (r_path in rasters_to_align) {
  r <- rast(r_path)
  
  # Align mask to this raster
  forest_aligned <- resample(forest, r, method = "near")  # nearest neighbor for categorical
  out_path <- sub("\\.tif$", "_aligned.tif", r_path)
  
  writeRaster(
    forest_aligned,
    out_path,
    overwrite = TRUE,
    datatype = "INT1U",
    gdal = c("COMPRESS=LZW", "BIGTIFF=YES", "TILED=YES")
  )
  
  message("✓ Mask aligned to: ", basename(r_path))
}
# Load rasters
forest <- rast("data/derived/resampled/forest_mask_30m_resampled.tif")
wildfire <- rast("data/derived/resampled/wildfire_id_30m_resampled.tif")

# Align mask to wildfire raster
forest_aligned <- resample(forest, wildfire, method = "near")  # nearest neighbor for categorical
writeRaster(
  forest_aligned,
  "data/derived/resampled/forest_mask_30m_resampled_aligned.tif",
  overwrite = TRUE,
  datatype = "INT1U",
  gdal = c("COMPRESS=LZW", "BIGTIFF=YES", "TILED=YES")
)

In [7]:
# ============================================================
# 6. Align mask to each raster
# ============================================================

# Load the original mask
forest <- rast("data/derived/resampled/forest_mask_30m_resampled.tif")

# List of rasters you want to align the mask to
rasters_to_align <- c(
  "data/derived/resampled/wildfire_id_30m_resampled.tif",
  "data/derived/resampled/biotic_gridded_1km_all_years_severity_30m_resampled.tif",
  "data/derived/resampled/hd_fingerprint_30m_resampled.tif",
  "data/derived/resampled/pdsi_annual_30m_resampled.tif"
)

for (r_path in rasters_to_align) {
  r <- rast(r_path)
  
  # Align mask to this raster
  forest_aligned <- resample(forest, r, method = "near")  # nearest neighbor for categorical
  out_path <- sub("\\.tif$", "_mask_aligned.tif", r_path)
  
  writeRaster(
    forest_aligned,
    out_path,
    overwrite = TRUE,
    datatype = "INT1U",
    gdal = c("COMPRESS=LZW", "BIGTIFF=YES", "TILED=YES")
  )
  
  message("✓ Mask aligned to: ", basename(r_path))
}

✓ Mask aligned to: wildfire_id_30m_resampled.tif



✓ Mask aligned to: biotic_gridded_1km_all_years_severity_30m_resampled.tif



✓ Mask aligned to: hd_fingerprint_30m_resampled.tif



✓ Mask aligned to: pdsi_annual_30m_resampled.tif

